In [1]:
from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [5]:
import numpy as np
from tqdm import tqdm
import math
import os

import tensorflow as tf

import tensorflow.keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
K.clear_session()
import itertools
from tensorflow.keras import layers, Model
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.densenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

import os
import cv2
from PIL import Image
import numpy as np

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input, decode_predictions

from keras.applications.densenet import DenseNet121

In [12]:
image_directory = '/content/gdrive/MyDrive/our_data/'
SIZE = 224
dataset = []   
label = []  


In [14]:
fire_images = os.listdir(image_directory + 'fire/')
for i, image_name in enumerate(fire_images):   
    
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'fire/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [15]:
non_fire_images = os.listdir(image_directory + 'non_fire/')
for i, image_name in enumerate(non_fire_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'non_fire/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [16]:
dataset = np.array(dataset)
label = np.array(label)
print(len(dataset))

1628


In [17]:
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)

In [19]:
from tensorflow.keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [20]:
IMAGE_SIZE = [SIZE, SIZE]

In [22]:
##VGG model      
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
       layer.trainable = False

folders = glob('/content/gdrive/MyDrive/our_data/*')

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='sigmoid')(x) # softmax

        # create a model object
model_vgg = Model(inputs=vgg.input, outputs=prediction)

        #model_vgg.summary()

model_vgg.compile(optimizer='adam',  # Good default optimizer to start with
                    loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
                      metrics=['accuracy'])  # what to track

history_vgg = model_vgg.fit(X_train, y_train, epochs=1)  # train the model

val_loss, val_acc = model_vgg.evaluate(X_test, y_test)  # evaluate the out of sample data with model

print("model loss is: ",val_loss)  # model's loss (error)

print(" Model accuracy is: ", val_acc)  # model's accuracy

11/11 [==============================] - 121s 11s/step - loss: 0.5704 - accuracy: 0.7761
model loss is:  0.5704332590103149
 Model accuracy is:  0.7760736346244812


In [23]:
#RESNET50
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


for layer in resnet.layers:
  layer.trainable = False

folders = glob('/content/gdrive/MyDrive/our_data/*')


x = Flatten()(resnet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)


model_ResNet = Model(inputs=resnet.input, outputs=prediction)

# model.summary()

model_ResNet.compile(optimizer='adam',  # Good default optimizer to start with
             loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track

history_ResNet= model_ResNet.fit(X_train, y_train, epochs=1)  # train the model

val_loss, val_acc = model_ResNet.evaluate(X_test, y_test)  # evaluate the out of sample data with model

print("model loss is: ",val_loss)  # model's loss (error)

print(" Model accuracy is: ", val_acc)  # model's accuracy

11/11 [==============================] - 36s 3s/step - loss: 1.6676 - accuracy: 0.7515
model loss is:  1.667637586593628
 Model accuracy is:  0.7515337467193604


In [24]:
efficientnet = EfficientNetB0(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in efficientnet.layers:
  layer.trainable = False

  folders = glob('/content/gdrive/MyDrive/our_data/*')

  x = Flatten()(efficientnet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)


model_Efficient = Model(inputs=efficientnet.input, outputs=prediction)


#model_Efficient.summary()

model_Efficient.compile(optimizer='adam',  # Good default optimizer to start with
             loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track

history_Efficient= model_ResNet.fit(X_train, y_train, epochs=1)  # train the model

val_loss, val_acc = model_Efficient.evaluate(X_test, y_test)  # evaluate the out of sample data with model

print("model loss is: ",val_loss)  # model's loss (error)

print(" Model accuracy is: ", val_acc)  # model's accuracy

11/11 [==============================] - 13s 970ms/step - loss: 0.5908 - accuracy: 0.7515
model loss is:  0.5907837748527527
 Model accuracy is:  0.7515337467193604


In [25]:
#DENSENET
densenet = DenseNet121(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in densenet.layers:
  layer.trainable = False

folders = glob('/content/gdrive/MyDrive/our_data/*')

x = Flatten()(densenet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

model_DenseNet = Model(inputs=densenet.input, outputs=prediction)

#model_DenseNet.summary()

model_DenseNet.compile(optimizer='adam',  # Good default optimizer to start with
              loss='sparse_categorical_crossentropy',# how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track
             
history_DenseNet = model_DenseNet.fit(X_train, y_train, epochs=1) 

val_loss, val_acc = model_DenseNet.evaluate(X_test, y_test)  # evaluate the out of sample data with model
print("model loss is: ",val_loss)  # model's loss (error)

print(" Model accuracy is: ", val_acc)  # model's accuracy

11/11 [==============================] - 35s 3s/step - loss: 0.3835 - accuracy: 0.8558
model loss is:  0.3834618628025055
 Model accuracy is:  0.8558282256126404


In [26]:
pred1=model_vgg.predict(X_test)
pred2=model_ResNet.predict(X_test)
pred3=model_Efficient.predict(X_test)
pred4=model_DenseNet.predict(X_test)

In [29]:
#Ensemble max voting

import math
import sys
#final_pred = np.array([])
final_pred = []
#print(len(X_test))
#sys.exit()

for i in range(0,len(X_test)):
    temp = []
    temp.append(np.argmax(pred1[i]))
    temp.append(np.argmax(pred2[i]))
    temp.append(np.argmax(pred3[i]))
    temp.append(np.argmax(pred4[i]))

    one= sum(temp)
    zero= 4 - one
    
    
   
    if one>=zero:
      final_pred.append(1)
    else:
      final_pred.append(0)

final_pred = np.array(final_pred)
